This is inference with submission notebook - full training notebook can be found here:
https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-train

# Versions
* V1 (V7 train notebook) 4-CV efficientunetb0 512x512 (LB .834)
* V2 (V8 train notebook) loss bce (LB .835)
* V3 (V9 train notebook) efficientunetb1 (CV .871, LB .830)
* V4 (V10 train notebook) efficientunetb4 (CV .874, LB .839)
* V5 (V12 train notebook) efficientunetb7 (CV .858, LB .835)
* V6 (V13 train notebook) efficientunetb4 (CV .877, LB .836)
* V7 (V14 train notebook) efficientunetb4 with overlapped train data, summing preds in inference (CV .879, LB .843)
* V8 (V14 train notebook) efficientunetb4  THRESHOLD=0.4, interpolation = cv2.INTER_AREA, rle_encode_less_memory (LB .846)
* V9 (V14 train notebook) efficientunetb4, MIN_OVERLAP = 300

# Refferences:
* https://www.kaggle.com/joshi98kishan/hubmap-keras-pipeline-training-inference
* https://www.kaggle.com/bguberfain/memory-aware-rle-encoding/
* https://www.kaggle.com/leighplt/pytorch-fcn-resnet50

# Parameters
Read parameteres from notebook output, actually only **DIM** is used:

In [ ]:
!nvidia-smi

In [ ]:
mod_path = '../input/unetkers/'
import yaml
import pprint
with open(mod_path+'params.yaml') as file:
    P = yaml.load(file, Loader=yaml.FullLoader)
    pprint.pprint(P)
    
THRESHOLD = 0.4
WINDOW = 1024
MIN_OVERLAP = 300
NEW_SIZE = P['DIM']

# Metrics

In [ ]:
import json

with open(mod_path + 'metrics.json') as json_file:
    M = json.load(json_file)
print('Model run datetime: '+M['datetime'])
print('OOF val_dice_coe: ' + str(M['oof_dice_coe']))

# Packages

In [ ]:
! pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index -q
! pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index -q
import numpy as np
import pandas as pd
import os
import glob
import gc

import rasterio
from rasterio.windows import Window

import pathlib
from tqdm.notebook import tqdm
import cv2

import tensorflow as tf
import efficientnet as efn
import efficientnet.tfkeras

# Functions

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

# Models

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
fold_models = []
for fold_model_path in glob.glob(mod_path+'*.h5'):
    print(fold_model_path)
    fold_models.append(tf.keras.models.load_model(fold_model_path,compile = False))
print(len(fold_models))

# Results

In [ ]:
p = pathlib.Path('../input/hubmap-kidney-segmentation')
subm = {}
if len(os.listdir('../input/hubmap-kidney-segmentation/test/'))!=10:
    for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):

        print(f'{i+1} Predicting {filename.stem}')

        dataset = rasterio.open(filename.as_posix(), transform = identity)
        slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
        preds = np.zeros(dataset.shape, dtype=np.uint8)

        for (x1,x2,y1,y2) in slices:
            image = dataset.read([1,2,3],
                        window=Window.from_slices((x1,x2),(y1,y2)))
            image = np.moveaxis(image, 0, -1)
            image = cv2.resize(image, (NEW_SIZE, NEW_SIZE),interpolation = cv2.INTER_AREA)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image = np.expand_dims(image, 0)

            pred = None

            for fold_model in fold_models:
                if pred is None:
                    pred = np.squeeze(fold_model.predict(image))
                else:
                    pred += np.squeeze(fold_model.predict(image))

            pred = pred/len(fold_models)

            pred = cv2.resize(pred, (WINDOW, WINDOW))
            preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)

        preds = (preds > 0.5).astype(np.uint8)

        subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
        #print(np.sum(preds))
        del preds
        gc.collect();


In [ ]:
# p = pathlib.Path('../input/hubmap-kidney-segmentation')
# subm = {}
# #if len(os.listdir('../input/hubmap-kidney-segmentation/test/'))!=10:
# if 1:
#     for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
#                             total = len(list(p.glob('test/*.tiff')))):

#         print(f'{i+1} Predicting {filename.stem}')

#         dataset = rasterio.open(filename.as_posix(), transform = identity)
#         slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
#         preds = np.zeros(dataset.shape, dtype=np.uint8)

#         for (x1,x2,y1,y2) in slices:
#             image = dataset.read([1,2,3],
#                         window=Window.from_slices((x1,x2),(y1,y2)))
#             image = np.moveaxis(image, 0, -1)
#             image = cv2.resize(image, (NEW_SIZE, NEW_SIZE),interpolation = cv2.INTER_AREA)
#             image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#             image = np.expand_dims(image, 0)

#             pred = None

#             for fold_model in fold_models:
#                 if pred is None:
#                     pred = np.squeeze(fold_model.predict(image))
#                 else:
#                     pred += np.squeeze(fold_model.predict(image))
        
#         pred = pred/len(fold_models)
        
#         pred = cv2.resize(pred, (WINDOW, WINDOW))
#         preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)
    
#     preds = (preds > 0.5).astype(np.uint8)
    
#     subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
#     #print(np.sum(preds))
#     del preds
#     gc.collect();

In [ ]:
open('../input/effnetmodels/params.yaml').readlines()

# Making submission

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission_1.csv', index=False)
submission.head()

In [ ]:
import numpy as np
import pandas as pd
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
def rle_encode_less_memory(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    This simplified method requires first and last pixel to be zero
    '''
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

def image_size_dict(img_id, x, y):
    image_id = [thing[:-5] for thing in img_id]
    x_y = [(x[i], y[i]) for i in range(0, len(x))]    
    return dict(zip(image_id, x_y))


def global_shift_mask(maskpred1, y_shift, x_shift):
    """
    applies a global shift to a mask by padding one side and cropping from the other
    """
    if y_shift <0 and x_shift >=0:
        maskpred2 = np.pad(maskpred1, [(0,abs(y_shift)), (abs(x_shift), 0)], mode='constant', constant_values=0)
        maskpred3 = maskpred2[abs(y_shift):, :maskpred1.shape[1]]
    elif y_shift >=0 and x_shift <0:
        maskpred2 = np.pad(maskpred1, [(abs(y_shift),0), (0, abs(x_shift))], mode='constant', constant_values=0)
        maskpred3 = maskpred2[:maskpred1.shape[0], abs(x_shift):]
    elif y_shift >=0 and x_shift >=0:
        maskpred2 = np.pad(maskpred1, [(abs(y_shift),0), (abs(x_shift), 0)], mode='constant', constant_values=0)
        maskpred3 = maskpred2[:maskpred1.shape[0], :maskpred1.shape[1]]
    elif y_shift < 0 and x_shift < 0:
        maskpred2 = np.pad(maskpred1, [(0, abs(y_shift)), (0, abs(x_shift))], mode='constant', constant_values=0)
        maskpred3 = maskpred2[abs(y_shift):, abs(x_shift):]
    return maskpred3

In [ ]:
dfpred = pd.read_csv('./submission_1.csv')
TARGET_ID = 'afa5e8098'
y_shift = -40
x_shift = -24

In [ ]:

dfinfo = pd.read_csv('../input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv')

size_dict = image_size_dict(dfinfo.image_file, dfinfo.width_pixels, dfinfo.height_pixels)  #dict which contains image sizes mapped to id's
mask_shape = size_dict.get(TARGET_ID)

taridx = dfpred[dfpred['id']==TARGET_ID].index.values[0]  #row of TARGET_ID in dfpred
maskpred = rle2mask(dfpred.iloc[taridx]['predicted'], mask_shape)

maskpred1 = maskpred.copy()
maskpred1[maskpred1>0]=1

In [ ]:
mask_shifted = global_shift_mask(maskpred1, y_shift, x_shift)  #apply specified shift to mask
newrle = rle_encode_less_memory(mask_shifted)  #rle encode shifted mask

dfpred.at[taridx, 'predicted'] = newrle


In [ ]:
dfsample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')

mydict = dict(zip(dfpred['id'], dfpred['predicted']))

dfsample['predicted'] = dfsample['id'].map(mydict).fillna(dfsample['predicted'])

dfsample = dfsample.replace(np.nan, '', regex=True)



In [ ]:
dfsample.to_csv('submission.csv',index=False)

In [ ]:
!ls ../input/unetmodels